# Just enough LangChain for RAG

I use OpenAI below. You may need to adjust for a different LLM using examples from week 4.

## Setting up Elasticsearch 

In [1]:
import os
from icecream import ic
from dotenv import load_dotenv
load_dotenv(".env", override=True)

from elasticsearch import Elasticsearch

es = None

if 'ELASTIC_CLOUD_ID' in os.environ:
  es = Elasticsearch(
    cloud_id=os.environ['ELASTIC_CLOUD_ID'],
    basic_auth=(os.environ['ELASTIC_USER'], os.environ['ELASTIC_PASSWORD']),
    request_timeout=30
  )
elif 'ELASTIC_URL' in os.environ:
  es = Elasticsearch(
    os.environ['ELASTIC_URL'],
    basic_auth=(os.environ['ELASTIC_USER'], os.environ['ELASTIC_PASSWORD']),
    request_timeout=30
  )
else:
  print("env needs to set either ELASTIC_CLOUD_ID or ELASTIC_URL")

if es:
    ic(es.info()['tagline']) # should return cluster info

ic| es.info()['tagline']: 'You Know, for Search'


## Using a langchain abstraction for ELSER

In [3]:
from langchain.vectorstores.elasticsearch import ElasticsearchStore

index_name = "elser_sotu_paragraphs"

es_elser = ElasticsearchStore(
    es_connection=es,
    index_name=index_name,
    strategy=ElasticsearchStore.SparseVectorRetrievalStrategy()
)

## Example of a simple semantic search

In [22]:
human_input = "what did Biden say about the Ukrainian people?"

best_result = ic(es_elser.similarity_search(human_input, k=1)[0])

ic| es_elser.similarity_search(human_input, k=1)[0]: Document(page_content='shake the very foundations of the free world, thinking he could make it bend to his menacing ways. But he badly miscalculated. He thought he could roll into Ukraine and the world would roll over. Instead, he met with a wall of strength he never anticipated or imagined. He met the Ukrainian people.From President Zelenskiy, their—to every Ukrainian, their fearlessness, their courage, their determination literally inspires the world. Groups of citizens blocking tanks with their bodies.Everyone from students to retirees, to teachers turned soldiers defending their homeland. And in this struggle—President Zelenskiy said in his speech to the European Parliament, "Light will win over darkness."The Ukrainian Ambassador to the United States is here tonight sitting with the First Lady. Let each of', metadata={'date': 'March 1, 2022', 'sotu_id': 'Biden-2022-03-01', 'date_iso': '2022-03-01', 'administration': 'Biden', 'chu

## Getting the LLM ready

This is likely the part you'll have to change if you are not using OpenAI

In [23]:
import openai
from langchain.chat_models import ChatOpenAI
def load_openai_llm():
    openai.api_key = os.environ["OPENAI_API_KEY"]
    default_model = "gpt-3.5-turbo"
    llm = ChatOpenAI(
        temperature=0.3,
        model=default_model
    )
    return llm

LLM=load_openai_llm()

In [24]:
LLM.predict("The prime minister of the United Kingdom is ")

'Boris Johnson.'

## LangChain PromptTemplate

LangChain prompts are templates in the following format, letting you bring your own variables retrieved from the structured and unstrutured documents we indexed into Elasticsearch

The following code composes a prompt that could be used against OpenAI

If you are using LLama2 or another LLM that requires special tokens to separate the system prompt, you'll need to add those yourself.

In [25]:
TEMPLATE = """You are a helpful AI Chatbot that answers questions 
about past Presidential State of the Union addresses using only the following provided context.
If you can't answer the question using the following context say "I don't know".

Context: 
On {date}, President {administration} said the following:
{context}

Human: {input}
AI:"""


from langchain.prompts.prompt import PromptTemplate

PROMPT = PromptTemplate(
    input_variables=["date", "administration", "context", "input"], 
    template=TEMPLATE
)

print(PROMPT.format(
    date= best_result.metadata["date"],
    administration= best_result.metadata["administration"],
    context= best_result.page_content,
    input= human_input
))

You are a helpful AI Chatbot that answers questions 
about past Presidential State of the Union addresses using only the following provided context.
If you can't answer the question using the following context say "I don't know".

Context: 
On March 1, 2022, President Biden said the following:
shake the very foundations of the free world, thinking he could make it bend to his menacing ways. But he badly miscalculated. He thought he could roll into Ukraine and the world would roll over. Instead, he met with a wall of strength he never anticipated or imagined. He met the Ukrainian people.From President Zelenskiy, their—to every Ukrainian, their fearlessness, their courage, their determination literally inspires the world. Groups of citizens blocking tanks with their bodies.Everyone from students to retirees, to teachers turned soldiers defending their homeland. And in this struggle—President Zelenskiy said in his speech to the European Parliament, "Light will win over darkness."The Ukrai

## Getting the LLM to work with the retrieved context

remember ```best_result``` shoulds still have the retrieved document from before.

In [26]:
from langchain.chains import LLMChain

conversation = LLMChain(
    prompt=PROMPT,
    llm=LLM,
    verbose=True
)

ai_response = conversation.run(
    date= best_result.metadata["date"],
    administration= best_result.metadata["administration"],
    context= best_result.page_content,
    input= human_input
)

print(ai_response)



> Entering new LLMChain chain...
Prompt after formatting:
You are a helpful AI Chatbot that answers questions 
about past Presidential State of the Union addresses using only the following provided context.
If you can't answer the question using the following context say "I don't know".

Context: 
On March 1, 2022, President Biden said the following:
shake the very foundations of the free world, thinking he could make it bend to his menacing ways. But he badly miscalculated. He thought he could roll into Ukraine and the world would roll over. Instead, he met with a wall of strength he never anticipated or imagined. He met the Ukrainian people.From President Zelenskiy, their—to every Ukrainian, their fearlessness, their courage, their determination literally inspires the world. Groups of citizens blocking tanks with their bodies.Everyone from students to retirees, to teachers turned soldiers defending their homeland. And in this struggle—President Zelenskiy said in his speech to the E